In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📉 Slowing Growth? Exploring Declining TFP Advancement

In recent decades, many advanced economies have experienced a slowdown in productivity growth compared to the mid-20th century. This has sparked debate about whether we are entering an era of **secular stagnation** or if the drivers of technological progress are weakening.

Potential reasons cited include:
* Ideas are getting harder to find ("fishing out" hypothesis).
* The impact of major past innovations (electricity, computers) is fading.
* Measurement issues might understate true productivity gains (especially from digital goods).
* Decreasing R&D intensity or effectiveness.

This simulation models one simple way this could occur: by assuming the **growth rate of Total Factor Productivity (TFP), $g_A$, declines linearly over time.**

# ⚙️ Modeling Declining TFP Growth

We modify the standard growth framework by making the TFP growth rate ($g_{A,t}$) time-dependent:

$$ g_{A,t} = \max(0, g_{A,0} - \text{decay} \times t) $$

-   $g_{A,t}$: TFP growth rate at time $t$.
-   $g_{A,0}$: Initial TFP growth rate at time $t=0$.
-   $\text{decay}$: A parameter determining how quickly the TFP growth rate falls each period.
-   $\max(0, ...)$: Ensures the growth rate doesn't become negative.

The level of TFP ($A_t$) then evolves according to:
$$ A_{t+1} = A_t (1 + g_{A,t}) $$

Output ($Y_t$) is still determined by a standard production function (e.g., Cobb-Douglas) using the current levels of $A_t$, $K_t$, and $L_t$. Capital ($K_t$) accumulates based on saving and depreciation as in the Solow model (assuming constant $s$ and $\delta$ here for simplicity). Labor ($L_t$) is assumed constant.

$$ Y_t = A_t K_t^{\alpha} L^{1 - \alpha} $$
$$ K_{t+1} = K_t + s Y_t - \delta K_t $$

The simulation shows how this declining TFP growth affects the paths of TFP itself, capital, total output, and the overall output growth rate ($g_Y$).

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def slowing_tfp_growth_model(A0=1.0, gA0=0.02, decay=0.0005,
                             alpha=0.33, s=0.2, delta=0.05,
                             K0=10.0, L0=100.0, T=100):
    """
    Simulates growth with declining TFP growth rate over time.
    gA(t) = max(0, gA0 - decay * t)
    Y = A * K^alpha * L^(1-alpha)
    K dynamics: K_dot = sY - delta*K (L is constant)

    Args:
        A0 (float): Initial TFP level.
        gA0 (float): Initial TFP growth rate.
        decay (float): Rate at which TFP growth declines per period.
        alpha (float): Capital share.
        s (float): Savings rate.
        delta (float): Depreciation rate.
        K0 (float): Initial Capital stock.
        L0 (float): Labor force (assumed constant).
        T (int): Time Horizon (number of periods).
    """
    # Ensure T is an integer
    T = int(T)
    if T < 2: T = 2

    # Input validation
    alpha = np.clip(alpha, 0.01, 0.99)
    s = np.clip(s, 0.01, 0.99)
    delta = max(delta, 0)
    gA0 = max(gA0, 0)
    decay = max(decay, 0)
    A0 = max(A0, 0.1)
    K0 = max(K0, 0.1)
    L0 = max(L0, 1)

    # Arrays to store results (size T+1 for levels)
    time = np.arange(T + 1)
    A = np.zeros(T + 1)
    K = np.zeros(T + 1)
    L = np.full(T + 1, L0) # Constant labor force
    Y = np.zeros(T + 1)
    g_A = np.zeros(T)     # TFP growth rate (length T)
    g_K = np.zeros(T)     # Capital growth rate
    g_Y = np.zeros(T)     # Output growth rate

    # Initial conditions
    A[0] = A0
    K[0] = K0

    # --- Simulation Loop (t=0 to T-1 to calculate state for t+1) ---
    for t in range(T):
        # Production in period t
        Y[t] = A[t] * K[t]**alpha * L[t]**(1 - alpha)

        # Calculate TFP growth for *this* period (used for next period's A)
        current_g_A = max(0, gA0 - decay * t)
        g_A[t] = current_g_A # Store the rate used from t to t+1

        # Calculate next period's state variables
        A[t+1] = A[t] * (1 + current_g_A)

        # Capital accumulation
        investment = s * Y[t]
        depreciation = delta * K[t]
        K[t+1] = K[t] + investment - depreciation
        K[t+1] = max(K[t+1], 0) # Ensure non-negative capital

        # Store growth rates (from t to t+1)
        if K[t] > 1e-9: g_K[t] = (K[t+1] / K[t]) - 1
        else: g_K[t] = 0
        # Calculate Y[T] for the last period
        if t == T-1:
             Y[T] = A[T] * K[T]**alpha * L[T]**(1 - alpha)
        if Y[t] > 1e-9: g_Y[t] = (Y[t+1] / Y[t]) - 1
        else: g_Y[t] = 0


    # --- Plotting ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.ravel()
    plot_time_g = time[:-1] # Time for growth rates (length T)

    # Plot 0: TFP Level and Growth Rate
    ax0b = axes[0].twinx() # Secondary axis for growth rate
    line0a = axes[0].plot(time, A, label='TFP Level A(t)', color='red', lw=2)
    line0b = ax0b.plot(plot_time_g, g_A*100, label='TFP Growth Rate gA(t)', color='darkred', lw=2, linestyle='--')
    axes[0].set_title("TFP Level and Declining Growth Rate")
    axes[0].set_xlabel("Time (t)")
    axes[0].set_ylabel("TFP Level A(t)", color='red')
    ax0b.set_ylabel("TFP Growth Rate gA(t) (%)", color='darkred')
    axes[0].tick_params(axis='y', labelcolor='red')
    ax0b.tick_params(axis='y', labelcolor='darkred')
    ax0b.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.2f}%'))
    ax0b.set_ylim(bottom=min(0, (g_A*100).min()*1.1)) # Adjust ylim for growth
    # Combine legends
    lines = line0a + line0b
    labels = [l.get_label() for l in lines]
    axes[0].legend(lines, labels, loc='best', fontsize='small')
    axes[0].grid(True, linestyle='--', alpha=0.6)


    # Plot 1: Capital Stock (Log Scale)
    axes[1].plot(time, K, label='Capital Stock K(t)', color='blue', lw=2)
    axes[1].set_yscale('log')
    axes[1].set_title('Capital Stock (Log Scale)')
    axes[1].set_xlabel("Time (t)")
    axes[1].set_ylabel("Capital K(t) (Log Scale)")
    axes[1].legend(fontsize='small')
    axes[1].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 2: Total Output (Log Scale)
    axes[2].plot(time, Y, label='Total Output Y(t)', color='black', lw=2)
    axes[2].set_yscale('log')
    axes[2].set_title('Total Output (Log Scale)')
    axes[2].set_xlabel("Time (t)")
    axes[2].set_ylabel("Output Y(t) (Log Scale)")
    axes[2].legend(fontsize='small')
    axes[2].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 3: Output Growth Rate
    axes[3].plot(plot_time_g, g_Y*100, label='Output Growth Rate gY(t)', color='black', lw=2)
    axes[3].plot(plot_time_g, g_A*100, label='TFP Growth Rate gA(t)', color='red', lw=1.5, linestyle='--')
    axes[3].set_title('Output Growth Rate vs TFP Growth Rate')
    axes[3].set_xlabel("Time (t)")
    axes[3].set_ylabel("Growth Rate (%)")
    axes[3].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.2f}%'))
    axes[3].legend(fontsize='small')
    axes[3].grid(True, linestyle='--', alpha=0.6)
    axes[3].axhline(0, color='grey', lw=0.5) # Zero growth line


    fig.suptitle(f"Simulation of Slowing Growth (gA declines by {decay:.4f} per year)", fontsize=16, y=1.03)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    # --- Display Final Growth Rates ---
    final_gY = g_Y[-1] if T > 0 else np.nan
    final_gA = g_A[-1] if T > 0 else np.nan
    final_gK = g_K[-1] if T > 0 else np.nan

    results_md = f"""
    ### 📉 Growth Results in Final Period (t={T-1}):

    * **TFP Growth (gA):** {final_gA:.3%}
    * **Capital Growth (gK):** {final_gK:.3%}
    * **Output Growth (gY):** {final_gY:.3%}

    *Note: As TFP growth declines, output growth also tends to slow down, potentially towards zero if TFP growth hits zero and the economy reaches a Solow-style steady state for K/L.*
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    slowing_tfp_growth_model,
    A0=FloatSlider(value=1.0, min=0.5, max=5.0, step=0.1, description='Initial TFP (A0):', style=style, layout=layout),
    gA0=FloatSlider(value=0.02, min=0.0, max=0.05, step=0.001, description='Initial TFP Growth (gA0):', style=style, layout=layout, readout_format='.2%'),
    decay=FloatSlider(value=0.0003, min=0.0, max=0.001, step=0.00005, description='TFP Growth Decay Rate:', style=style, layout=layout, readout_format='.4f'),
    alpha=FloatSlider(value=0.33, min=0.1, max=0.8, step=0.01, description='Capital Share (alpha α):', style=style, layout=layout, readout_format='.2f'),
    s=FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='Savings Rate (s):', style=style, layout=layout, readout_format='.2f'),
    delta=FloatSlider(value=0.05, min=0.01, max=0.15, step=0.005, description='Depreciation (delta δ):', style=style, layout=layout, readout_format='.3f'),
    K0=FloatSlider(value=10.0, min=1, max=100.0, step=1, description='Initial Capital (K0):', style=style, layout=layout, readout_format='.0f'),
    L0=FloatSlider(value=100.0, min=10, max=1000, step=10, description='Labor (L0, constant):', style=style, layout=layout, readout_format='.0f'),
    T=IntSlider(value=100, min=20, max=300, step=10, description='Time Periods (T):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=1.0, description='Initial TFP (A0):', layout=Layout(width='95%'), max=…

# 📉 Model 9: Slowing Growth in the 21st Century

This model simulates how declining TFP growth over time can cause a **slowdown in output**.

# Equation:

$begin:math:display$
Y_t = A_t \\cdot K_t^{\\alpha} \\cdot L_t^{1 - \\alpha}, \\quad \\text{where } A_{t+1} = A_t (1 + g_t)
$end:math:display$

- $begin:math:text$ g_t = g_0 - \\text{decay} \\cdot t $end:math:text$
- This captures the **decline in idea productivity**, **saturation of innovation**, or **structural stagnation**.

> Use this to visualize concerns about long-run growth deceleration and secular stagnation.


Ready to review or revisit any model? Or shall we begin **site-wide integration / beautification / deployment prep**?